## Julia: optimize simulation of simple dynamical system
Source: [user45893 at StackOverflow](https://stackoverflow.com/questions/32845996/julia-optimize-simulation-of-simple-dynamical-system) Sept. 2015

In [1]:
# Pkg.add("Devectorize")

In [2]:
using Devectorize # Julia framework, which provides macros and functions to de-vectorize a vector expression. 
# Description: (https://github.com/lindahua/Devectorize.jl)
# With Devectorize, users can write computations in high-level vectorized way and at the same time enjoying the high run-time performance of de-vectorized loops. 
# Devectorize" "automatically translates vectorized expressions into faster tight-loops, which often results in 2x - 8x performance gain."
# Commenter suggests that this is not the case, and better to just write loops if speed is critical

INFO: Precompiling module Devectorize.

Use "abstract type FunKind end" instead.

Use "abstract type TMode end" instead.

Use "abstract type TExpr end" instead.

Use "abstract type TEWise<:TExpr end" instead.

Use "abstract type TGeneralVar<:TEWise end" instead.

Use "abstract type TScalar<:TEWise end" instead.

Use "abstract type TRef<:TEWise end" instead.

Use "const TIndex = Union{Int,Symbol}" instead.

Use "abstract type TRange end" instead.

Use "const TFunCall = Union{TMap,TReduc,TColwiseReduc,TRowwiseReduc}" instead.

Use "const TRValue = Union{TEWise,TFunCall}" instead.

Use "const TLValue = Union{TGeneralVar,TRef}" instead.
  likely near /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:293
  likely near /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:293
  likely near /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:293

Use "abstract

In [3]:
function train_network(A, T, Of, cs, dt)
    N, I = size(T)
    z    = zeros(I)
    r    = zeros(N)

    @inbounds for t in 1:size(cs, 1)  #  skip such bounds checks for speed/SIMD (https://docs.julialang.org/en/latest/devdocs/boundscheck/)
        # precompute
        Az  = A*z
        Ofr = Of*r

        # compute training signal
        @devec z += dt.*(Az + cs[t] - 0.5.*z)
        I_teach   = T*(Az + cs[t])
        Tz        = T*z

        # rate updates
        @devec r += dt.*(I_teach - Ofr - 0.1.*r)

        # weight updates
        for i in 1:I
            @devec T[:, i] += dt.*1e-3.*(z[i].*r - T[:, i])
        end

        for n in 1:N
            @devec Of[:, n] += dt.*1e-3.*(Tz.*r[n] - Of[:, n])     
        end
    end
end

  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Devectorize.TExpr}, ::Int64) at ./deprecated.jl:57
 [3] check_funcall_args(::Devectorize.TVar, ::Vararg{Devectorize.TExpr,N} where N) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:416
 [4] tcall(::Symbol, ::Array{Devectorize.TEWise,1}) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:463
 [5] tcall(::Expr) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:496
 [6] texpr(::Expr) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Devectorize/src/texpr.jl:607
 [7] _collect(::Array{Any,1}, ::Base.Generator{Array{Any,1},Devectorize.#texpr}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:454
 [8] tcall(::Expr) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resou

train_network (generic function with 1 method)

In [4]:
# init parameters
N, I = 20, 2
dt  = 1e-3

# init weights
T = rand(N, I)*N
A = rand(I, I)
Of = rand(N, N)/N

# simulation time & input
sim_T = 2000
ts = 0:dt:sim_T
cs = randn(size(ts, 1), I)

2000001×2 Array{Float64,2}:
  0.590297    -1.3061   
  0.372789     1.56986  
  0.946243     0.924054 
 -0.759128     0.341176 
  0.924678    -2.55936  
  0.00770045  -1.2796   
 -1.03795     -0.679051 
 -0.687983     0.885255 
  0.605628     0.435501 
 -0.448274     0.280454 
  0.130458     0.502419 
  0.21645     -0.509237 
 -0.966325     0.133707 
  ⋮                     
  0.860585    -0.199513 
 -0.885967    -0.823579 
  0.0974658   -0.140565 
  0.593716     0.179675 
 -0.480358    -0.257579 
 -0.11323      0.0543308
 -0.472055    -0.131496 
  0.916498    -0.0729267
 -0.485756    -0.329036 
 -0.231642     0.666361 
 -0.670648    -1.51886  
 -0.237538    -0.421701 

Test: Timing the network (2.000.000 steps) with use of @devec

In [5]:
@time train_network(A, T, Of, cs, dt)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Tuple{Int64}) at ./deprecated.jl:57
 [3] train_network(::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}, ::Float64) at ./In[3]:12
 [4] include_string(::String, ::String) at ./loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/IJulia/src/execute_request.jl:160
 [6] eventloop(::ZMQ.Socket) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##11#14)() at ./task.jl:335
while loading In[5], in expression starting on line 237
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Tuple{Int64}) at ./deprecated.jl:57
 [3] train_network(::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}, ::Float64) at ./In[3]:17
 [4] include_string(::String, ::String)

LoadError: [91mInterruptException:[39m

yields the timings

3.420486 seconds (26.12 M allocations: 2.299 GB, 6.65% gc time)

### Update 1

Following the advice by David Sanders I got rid of the devec macro and wrote out the loops. This indeed reduces array allocations and boosts performance by about 25%, here are the new numbers:

2.648113 seconds (18.00 M allocations: 1.669 GB, 5.60% gc time)
The smaller the network size, the bigger the boost. A gist of the updated simulation code can be found here.

### Update 2

Much of the memory allocations are due to the matrix-vector products. So, in order to get rid of those I replaced those products by an in-place BLAS operation, BLAS.genv!, which reduces timings by another 25% and memory allocation by 90%,

1.990031 seconds (2.00 M allocations: 152.589 MB, 0.69% gc time)
Updated code here.

### Update 3

The largest rank-1 update can also be replaced by two calls to in-place BLAS functions, namely BLAS.scal! for scaling and BLAS.ger! for a rank-1 update. The caveat is that both calls are fairly slow if more then one thread is used (problem with OpenBLAS?), so it is best to set

blas_set_num_threads(1)
There is a 15% gain in timings for a network size of 20, and a gain of 50% for a network of size 50. There are no more memory allocations, and the new timings are

1.638287 seconds (11 allocations: 1.266 KB)
Again, the updated code can be found here.

### Update 4

I wrote a basic Cython script to compare the results so far. The main difference is that I do not use any calls to BLAS but have loops: Injecting low-level BLAS calls is a pain in Cython, and calls to numpy dot have too much overhead for small network sizes (I tried...). Timings are

CPU times: user 3.46 s, sys: 6 ms, total: 3.47 s, Wall time: 3.47 s
which is roughly the same as the original version (from which, so far, 50% is shaved off).

In [42]:
r    = zeros(N)
z    = zeros(I)
N, I = size(T)
size(Of)

(20,20)

In [44]:
function train_network1(A, T, Of, cs, dt)
    N, I = size(T)
    z    = zeros(I)
    r    = zeros(N)

    @inbounds for t in 1:size(cs, 1)  #  skip such bounds checks for speed/SIMD (https://docs.julialang.org/en/latest/devdocs/boundscheck/)
        # precompute
        Az  = A*z
        Ofr = Of*r

        # compute training signal
        # @devec z += dt.*(Az + cs[t] - 0.5.*z)
        for i in 1:I
            z[i] += dt*(Az[i] + cs[t,i] - 0.5.*z[i])
        end
            
        I_teach   = T*(Az + cs[t])
        Tz        = T*z

        # rate updates
        # @devec r += dt.*(I_teach - Ofr - 0.1.*r)
        for i in 1:N
            r[i] += dt*(I_teach[i] - Ofr[i] - 0.1.*r[i])
        end

        # weight updates
        for i in 1:I
            # @devec T[:, i] += dt.*1e-3.*(z[i].*r - T[:, i])
            for j in 1:N
                T[j, i] += dt*1e-3*(z[i].*r[j] - T[j, i])
            end
        end

        for n in 1:N
            # @devec Of[:, n] += dt.*1e-3.*(Tz.*r[n] - Of[:, n])
            for j in 1:N
                Of[j, n] += dt*1e-3*(Tz[j]*r[n] - Of[j, n])
            end
        end
    end
end

train_network1 (generic function with 1 method)

In [45]:
@time train_network1(A, T, Of, cs, dt)

 33.031152 seconds (18.03 M allocations: 1.670 GB, 0.44% gc time)
